### Setting paths and imports

In [1]:
import sys
import os
import pandas as pd

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)
module_path

'/NFSHOME/mprado/AIIM/GRETEL/main2/GRETEL'

In [2]:
os.chdir(module_path)

In [3]:
# from src.evaluation.evaluator_manager import EvaluatorManager
# from src.evaluation.future.evaluator_manager_do import EvaluatorManager as PairedEvaluatorManager
from src.utils.context import Context
from src.data_analysis.future.data_analyzer import DataAnalyzer as data_analyzer

from src.evaluation.future.evaluator_manager_triplets import EvaluatorManager

In [4]:
# config_f_name = 'ensembles/triplets/TCR-TCO/E10-TCR-128-28-0.25_TCO_Ensemble-Bidirectional.jsonc'
# config_f_name = 'ensembles/triplets/TCR-TCO/E10-TCR-128-28-0.25_TCO_RSGG.jsonc'

# config_f_name = 'ensembles/triplets/TCR-TCO/il-explainer-selector.jsonc'
# config_f_name = 'ensembles/triplets/TCR-TCO/dl-explainer-selector.jsonc'

# config_f_name = 'ensembles/triplets/imdb/iRand.jsonc'
config_f_name = 'ensembles/imdb/base_explainers_t0/base_explainers_t0_fold0.json'

### Evaluating the explainer

In [8]:
config_path = os.path.join(module_path, 'lab', 'config', config_f_name)
runno = 1
    
print(f"Generating context for: {config_path}")
context = Context.get_context(config_path)
context.run_number = runno

context.logger.info(f"Executing: {context.config_file} Run: {context.run_number}")
context.logger.info("Creating the evaluation manager....................................")

context.logger.info("Creating the evaluators......................................................")
eval_manager = EvaluatorManager(context)

Generating context for: /NFSHOME/mprado/AIIM/GRETEL/main2/GRETEL/lab/config/ensembles/imdb/base_explainers_t0/base_explainers_t0_fold0.json
2024-08-13 22:48:43,1300237811 | INFO | 3960434 - Executing: /NFSHOME/mprado/AIIM/GRETEL/main2/GRETEL/lab/config/ensembles/triplets/imdb/iRand.jsonc Run: 1
2024-08-13 22:48:43,1300237824 | INFO | 3960434 - Creating the evaluation manager....................................
2024-08-13 22:48:43,1300237850 | INFO | 3960434 - Creating the evaluators......................................................
2024-08-13 22:48:43,1300238014 | INFO | 3960434 - Loading: IMDB-BINARY-607bc225e365745075c10dc6732061e7
2024-08-13 22:48:44,1300238329 | INFO | 3960434 - Created: IMDB-BINARY-607bc225e365745075c10dc6732061e7
2024-08-13 22:48:44,1300238331 | INFO | 3960434 - Instantiating: src.oracle.nn.gcn.DownstreamGCN
2024-08-13 22:48:44,1300238335 | INFO | 3960434 - Instantiating: torch.optim.RMSprop
2024-08-13 22:48:44,1300238339 | INFO | 3960434 - Instantiating: tor

In [17]:
evaluator = eval_manager._evaluators[0]

n_nodes = [inst.data.shape[0] for inst in evaluator.dataset.instances]
max(n_nodes)

136

In [5]:
config_path = os.path.join(module_path, 'lab', 'config', config_f_name)
runno = 1
    
print(f"Generating context for: {config_path}")
context = Context.get_context(config_path)
context.run_number = runno

context.logger.info(f"Executing: {context.config_file} Run: {context.run_number}")
context.logger.info("Creating the evaluation manager....................................")

context.logger.info("Creating the evaluators......................................................")
eval_manager = EvaluatorManager(context)

context.logger.info(
    "Evaluating the explainers............................................................."
)
eval_manager.evaluate()

Generating context for: /NFSHOME/mprado/AIIM/GRETEL/main2/GRETEL/lab/config/ensembles/triplets/imdb/iRand.jsonc
2024-08-13 22:44:31,1299985365 | INFO | 3960434 - Executing: /NFSHOME/mprado/AIIM/GRETEL/main2/GRETEL/lab/config/ensembles/triplets/imdb/iRand.jsonc Run: 1
2024-08-13 22:44:31,1299985377 | INFO | 3960434 - Creating the evaluation manager....................................
2024-08-13 22:44:31,1299985388 | INFO | 3960434 - Creating the evaluators......................................................
2024-08-13 22:44:31,1299985855 | INFO | 3960434 - Loading: IMDB-BINARY-607bc225e365745075c10dc6732061e7
2024-08-13 22:44:32,1299986792 | INFO | 3960434 - Created: IMDB-BINARY-607bc225e365745075c10dc6732061e7
2024-08-13 22:44:32,1299986857 | INFO | 3960434 - Instantiating: src.oracle.nn.gcn.DownstreamGCN
2024-08-13 22:44:32,1299987286 | INFO | 3960434 - Instantiating: torch.optim.RMSprop
2024-08-13 22:44:32,1299987287 | INFO | 3960434 - Instantiating: torch.nn.CrossEntropyLoss
2024-

KeyboardInterrupt: 

### Aggregating the stats

In [5]:
results_path = os.path.join(module_path, 'lab', 'output', 'results')
stats_file_path = os.path.join(module_path, 'lab', 'stats', 'results.csv')
res = data_analyzer.create_aggregated_dataframe(results_path)
res.to_csv(stats_file_path)
res

TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95f8bca65
TreeCyclesRand-cb519bb4e5768b2dc9cc04b95

,scope,dataset,oracle,explainer,Runtime,Runtime-std,GraphEditDistance,GraphEditDistance-std,Correctness,Correctness-std,OracleCalls,OracleCalls-std,OracleAccuracy,OracleAccuracy-std,Sparsity,Sparsity-std,Fidelity,Fidelity-std,Instability,Instability-std
0,xai-ens-asd,ASD-ca8cc36049eb8589f68c35b0d8eebfbb,ASDOracle-74012801e4bd624973d6c15c0a46f2eb,ExplanationMultiCriteriaAggregator,211.657759,35.730723,8.911818,2.667819,1.000000,0.000000,5812.985455,583.287964,0.773636,0.104798,0.020093,0.006049,0.547273,0.209596,1.002813,0.001543
1,xai-ens-asd,ASD-ca8cc36049eb8589f68c35b0d8eebfbb,ASDOracle-74012801e4bd624973d6c15c0a46f2eb,ExplanationStochasticAggregator,228.281466,52.532883,91.245909,15.527264,1.000000,0.000000,5761.752727,501.545682,0.773636,0.104798,0.205615,0.035011,0.547273,0.209596,0.972166,0.016495
2,xai-ens-asd,ASD-ca8cc36049eb8589f68c35b0d8eebfbb,ASDOracle-74012801e4bd624973d6c15c0a46f2eb,ExplanationIntersection,219.533001,33.331162,0.000000,0.000000,0.000000,0.000000,5817.239091,575.852626,0.773636,0.104798,0.000000,0.000000,0.000000,0.000000,0.998999,0.000069
3,xai-ens-asd,ASD-ca8cc36049eb8589f68c35b0d8eebfbb,ASDOracle-74012801e4bd624973d6c15c0a46f2eb,ExplanationBidirectionalSearch,267.091272,45.491886,13.030714,4.479636,0.574545,0.116831,6267.499091,499.526945,0.773636,0.104798,0.029410,0.010176,0.574545,0.116831,1.001414,0.001738
4,xai-ens-asd,ASD-ca8cc36049eb8589f68c35b0d8eebfbb,ASDOracle-74012801e4bd624973d6c15c0a46f2eb,ExplanationFrequency,232.174510,49.096034,0.250000,0.512348,0.019091,0.038236,5719.732727,548.221054,0.773636,0.104798,0.000561,0.001149,-0.019091,0.038236,0.981640,0.005303
5,xai-ens-asd,ASD-ca8cc36049eb8589f68c35b0d8eebfbb,ASDOracle-74012801e4bd624973d6c15c0a46f2eb,ExplanationRandom,243.448140,45.531990,0.150000,0.229129,0.029091,0.044499,5947.395455,623.230466,0.773636,0.104798,0.000338,0.000516,-0.009091,0.052381,0.999003,0.000025
6,xai-ens-asd,ASD-ca8cc36049eb8589f68c35b0d8eebfbb,ASDOracle-74012801e4bd624973d6c15c0a46f2eb,ExplanationUnion,232.680348,47.044103,217.339861,44.427534,0.940909,0.065839,5794.040000,517.171256,0.773636,0.104798,0.489851,0.100952,0.488182,0.209724,0.931670,0.041226
7,xai-ens-asd,ASD-ca8cc36049eb8589f68c35b0d8eebfbb,ASDOracle-74012801e4bd624973d6c15c0a46f2eb,ExplanationTopSelect,212.847775,41.799704,9.009091,2.492063,1.000000,0.000000,5728.440000,628.639048,0.773636,0.104798,0.020308,0.005645,0.547273,0.209596,1.002743,0.001361
8,xai-ens-asd,ASD-a90450198d54af7520c0a579af443e89,ASDOracle-12b3e9daa577d8245fb63e12b30ea64d,IRandExplainer,3.515603,0.746095,100.470794,35.749031,0.654545,0.106406,1146.750909,208.121988,0.770909,0.111103,0.226500,0.080482,0.236364,0.152825,1.098501,0.016209
9,xai-ens-asd,ASD-a90450198d54af7520c0a579af443e89,ASDOracle-12b3e9daa577d8245fb63e12b30ea64d,RSGG,65.319065,14.375991,350.047857,25.414409,0.466364,0.129337,606.255455,90.925438,0.770909,0.111103,0.788611,0.057844,0.226364,0.176019,0.863959,0.038371


### Visualizing individual instances

In [1]:
evaluator = eval_manager.evaluators[0]
evaluator

NameError: name 'eval_manager' is not defined

In [ ]:
exp = evaluator.explanations[2]
og_inst = exp.input_instance
cf_inst = exp.counterfactual_instances[0]

In [ ]:

changes = data_analyzer.get_cf_changes(og_inst, cf_inst, False)

In [ ]:
common_edges = changes['common edges']
added_edges = changes['added edges']
removed_edges = changes['removed edges']
common_nodes = changes['common nodes']
added_nodes = changes['added nodes']
removed_nodes = changes['removed nodes']

print(f'added edges: {added_edges}')
print(f'removed_edges: {removed_edges}')
print(f'added nodes: {added_nodes}')
print(f'removed nodes: {removed_nodes}')

In [ ]:
pos = data_analyzer.draw_graph(og_inst)

In [ ]:
data_analyzer.draw_counterfactual_actions(og_inst, cf_inst, position=pos)